# Prism SDK

# Crypto

Add this code to your `build.gradle` to install the Crypto module:

## SHA256 hash

The Crypto module provides a [SHA256](https://en.wikipedia.org/wiki/SHA-2) hash function that can be used as follows:

```kotlin
import io.iohk.atala.prism.crypto.Sha256

Sha256.compute(byteArrayOf(0))
```

## Elliptic-curve cryptography

This module includes low-level tools to work with public-key cryptography based on [elliptic curves](https://en.wikipedia.org/wiki/Elliptic-curve_cryptography).

Some helpful globals can be found in `ECConfig`:
```kotlin
import io.iohk.atala.prism.crypto.ECConfig

ECConfig.CURVE_NAME
ECConfig.PRIVATE_KEY_BYTE_SIZE
ECConfig.PUBLIC_KEY_BYTE_SIZE
```

`EC` provides access to key generation, signing and verification:
```kotlin
import io.iohk.atala.prism.crypto.EC

val keyPair = EC.generateKeyPair()
val privateKey = keyPair.privateKey
val publicKey = keyPair.publicKey

val data = byteArrayOf(1)
val dataSignature = EC.signBytes(data, privateKey)
EC.verifyBytes(data, publicKey, dataSignature)

val text = "PRISM"
val textSignature = EC.signText(text, privateKey)
EC.verifyText(text, publicKey, textSignature)
```

Keys can also be encoded and decoded as follows:
```kotlin
val privateKeyEncoded = privateKey.getEncoded()
val publicKeyEncoded = publicKey.getEncoded()

EC.toPrivateKeyFromBytes(privateKeyEncoded) == privateKey
EC.toPublicKeyFromBytes(publicKeyEncoded) == publicKey
```

## Key Derivation

You can use this module to derive keys from a given seed.

First, there are some utilities to work with mnemonic codes:
```kotlin
import io.iohk.atala.prism.crypto.derivation.KeyDerivation

val mnemonicCode = KeyDerivation.randomMnemonicCode()
// val mnemonicCode = MnemonicCode(mySeedPhrase)
KeyDerivation.getValidMnemonicWords().take(10)
KeyDerivation.isValidMnemonicWord("airport")
```

After obtaining a mnemonic code, you can generate a mnemonic seed and a root key:
```kotlin
val seed = KeyDerivation.binarySeed(mnemonicCode, "my_secret_password")
val extendedKey = KeyDerivation.derivationRoot(seed)

// An extended key contains the private and public keys inside as well as the
// derivation path used to obtain the key
extendedKey.keyPair()
extendedKey.privateKey()
extendedKey.publicKey()
extendedKey.path()
```

This module also supports derivation paths, so you can derive children keys:
```kotlin
import io.iohk.atala.prism.crypto.derivation.DerivationPath

val path = DerivationPath.fromPath("m/0'/0'/1'")
KeyDerivation.deriveKey(seed, path)
```

## Merkle Tree

It is possible to build and verify [Merkle tree](https://en.wikipedia.org/wiki/Merkle_tree) proofs of inclusion by using `MerkleTree`:
```kotlin
import io.iohk.atala.prism.crypto.*

val hash1 = Sha256.compute(byteArrayOf(0))
val hash2 = Sha256.compute(byteArrayOf(1))

val (root, proofs) = generateProofs(listOf(hash1, hash2))
val (proof1, proof2) = proofs

verifyProof(root, proof1)
verifyProof(root, proof2)
```

## Identity
Add this code to your `build.gradle` to install the Crypto module:

## Decentralized Identifiers

This module provides the necessary tools to work with [DIDs](https://en.wikipedia.org/wiki/Decentralized_identifiers).

All DIDs start with the URI scheme identifier `did`, and PRISM's DIDs are no exception. They also have a specific [DID method](https://www.w3.org/TR/did-core/#dfn-did-methods) attached to them:
```kotlin
import io.iohk.atala.prism.crypto.Sha256Digest
import io.iohk.atala.prism.identity.*
import io.iohk.atala.prism.protos.util.Base64Utils

PrismDid.PRISM_METHOD
```

There can be two forms of method-specific identifiers after the PRISM prefix: canonical and long. You can build both by using these utility methods:
```kotlin
val stateHash = Sha256Digest.fromHex("0f753f41e0f3488ba56bd581d153ae9b3c9040cbcc7a63245b4644a265eb3b77")
val encodedState = Base64Utils.decode("CmEKXxJdCgdtYXN0ZXIwEAFCUAoJc2VjcDI1NmsxEiAel_7KEiez4s_e0u8DyJwLkUnVmUHBuWU-0h01nerSNRohAJlR51Vbk49vagehAwQkFvW_fvyM1qa4ileIEYkXs4pF")

val canonical = PrismDid.buildCanonical(stateHash)
val long = PrismDid.buildLongForm(stateHash, encodedState)
```

You can also check whether a DID is of a specific form with this code:
```kotlin
canonical is CanonicalPrismDid
long is LongFormPrismDid
```

You can create a simple DID consisting of a single master key with this code:
```kotlin
import io.iohk.atala.prism.crypto.EC
import io.iohk.atala.prism.identity.PrismDid

// Set up your public key
val masterKey = EC.generateKeyPair().publicKey

PrismDid.buildLongFormFromMasterPublicKey(masterKey)
```

## Credentials
To install the module add the following to your `build.gradle`:

## Verifiable Credentials (VCs)

This module provides the necessary tools to work with [Verifiable Credentials - VCs](https://en.wikipedia.org/wiki/Verifiable_credentials).

Add these imports to run the examples listed on this module:

```kotlin
// necessary to sign a credential
import io.iohk.atala.prism.crypto.* 
import io.iohk.atala.prism.credentials.*
import io.iohk.atala.prism.credentials.content.*
import io.iohk.atala.prism.credentials.json.*
import kotlinx.datetime.*

// necessary to construct the credential content
import kotlinx.serialization.json.* 

```
Follow these steps to create, sign, and verify a credential.

1. Define the credential's content:

```kotlin
val credentialContent = CredentialContent(
    JsonObject(
        mapOf(
            Pair("credentialSubject", JsonObject(
                mapOf(
                    Pair("name", JsonPrimitive("José López Portillo")),
                    Pair("certificate", JsonPrimitive("Certificate of PRISM SDK tutorial completion"))
                )
            ))
        )
    )
)
```

2. Create the `Credential` object:

```kotlin
val credential = JsonBasedCredential(credentialContent)
```

3. You can sign a credential by using the `Crypto` module:

```kotlin
val masterKeyPair = EC.generateKeyPair()
val signedCredential = credential.sign(masterKeyPair.privateKey)
```

4. Your credential is ready and can be included in a batch now

## Node API
To install the module add the following to your `build.gradle`:

Add these imports to run the examples listed on this module:

```kotlin
import io.iohk.atala.prism.api.*
import io.iohk.atala.prism.api.models.AtalaOperationId
import io.iohk.atala.prism.api.models.AtalaOperationStatus
import io.iohk.atala.prism.api.node.*
// We need this to derive new keys
import io.iohk.atala.prism.crypto.derivation.KeyDerivation 
import io.iohk.atala.prism.identity.*
import kotlinx.coroutines.*
// Json is needed for defining credentials
import kotlinx.serialization.json.JsonObject 
import kotlinx.serialization.json.JsonPrimitive
```

## NodePayloadGenerator

_NodePayloadGenerator_ has convenient methods for generating API requests to the _PRISM Node_ service.
The constructor takes a _DID_ in long-form and a _Map_ of keys.

__NOTE:__ _NodePayloadGenerator_ stores private keys inside, so a user should create a new instance whenever they need to use new keys.

For example, following these steps, you can construct a new _DID_ creation request
1. Generate keys using a mnemonic code and password
```kotlin
val mnemonic = KeyDerivation.randomMnemonicCode()
val pass = "password"
val seed = KeyDerivation.binarySeed(mnemonic, pass)
val masterKeyPair = KeyGenerator.deriveKeyFromFullPath(seed, 0, MasterKeyUsage, 0)
val issuingKeyPair = KeyGenerator.deriveKeyFromFullPath(seed, 0, IssuingKeyUsage, 0)
```

2. Create a new _DID_ with the master key
```kotlin
val unpublishedDid =
    PrismDid.buildLongFormFromMasterPublicKey(masterKeyPair.publicKey)
```   

3. Create a new instance of _NodePayloadGenerator_
```kotlin
val nodePayloadGenerator = NodePayloadGenerator(
    unpublishedDid,
    mapOf(PrismDid.DEFAULT_MASTER_KEY_ID to masterKeyPair.privateKey)
)
```

4. Generate payload of the API request for new _DID_ creation
```kotlin
val createDidInfo = nodePayloadGenerator.createDid()
```

_createDidInfo_ now contains two objects:

- `payload: PrismPayload` is an object with an actual bytes we'll need to send
- `operationHash: Sha256Digest` Sha256 digest of an operation. We need this because `updateDid` operations are bounded sequentially using hashes.

## Node API

_NodeAuthApi_ provides the necessary abstractions to interact with _PRISM Node_.

To create an instance of this class, you need to specify the endpoint of the working _PRISM Node_ service.
```kotlin
val nodeAuthApi = NodeAuthApiImpl(GrpcOptions("https", "ppp.atalaprism.io", 50053))
```

In the [previous section](#nodepayloadgenerator) you've created an API request stored as _PrismPayload_.
This snippet actually sends request to the Node
```kotlin
val createDidOperationId = runBlocking {
    nodeAuthApi.createDid(
        createDidInfo.payload,
        unpublishedDid,
        PrismDid.DEFAULT_MASTER_KEY_ID
    )
}
```

Now you can use `createDidOperationId` to track the status of an operation using 
```kotlin
var createDidStatus = runBlocking { nodeAuthApi.getOperationStatus(createDidOperationId) }
```

### Verifying credentials

The following steps will create and verify credentials using Node API.

1. Create an object with credentials for a _holder_ identified by _holderUnpublishedDid_
```kotlin
val credentialClaim = CredentialClaim(
    subjectDid = holderUnpublishedDid,
    content =
        JsonObject(
            mapOf(
                Pair("name", JsonPrimitive("José López Portillo")),
                Pair("certificate", JsonPrimitive("Certificate of PRISM SDK tutorial completion"))
            )
        )
)
```

2. Create a new instance of _NodePayloadGenerator_ out of _nodePayloadGenerator_ from the previous section by adding the issuing key:
```kotlin
nodePayloadGenerator = NodePayloadGenerator(
    nodePayloadGenerator.did,
    nodePayloadGenerator.keys +
            (PrismDid.DEFAULT_ISSUING_KEY_ID to issuingKeyPair.privateKey)
)
```

3. Create a new request which issues a batch of credentials. The DID document corresponding to the issuer should have a key with id `PrismDid.DEFAULT_ISSUING_KEY_ID`.
```kotlin
val issueCredentialsInfo = nodePayloadGenerator.issueCredentials(
    PrismDid.DEFAULT_ISSUING_KEY_ID,
    arrayOf(credentialClaim)
)
```

4. Extract the signed credentials and the corresponding Merkle tree as a proof
```kotlin
val signedCredential = issueCredentialsInfo.credentialsAndProofs.first().signedCredential
val credentialMerkleProof = issueCredentialsInfo.credentialsAndProofs.first().inclusionProof
```

5. Send the issuing request to the Node
```kotlin
val issueCredentialBatchOperationId = runBlocking {
    nodeAuthApi.issueCredentials(
        issueCredentialsInfo.payload,
        issuerDid,
        PrismDid.DEFAULT_ISSUING_KEY_ID,
        issueCredentialsInfo.merkleRoot
    )
}
```

6. Wait until Node applies the operation
```kotlin
var status = runBlocking {
    nodePublicApi.getOperationStatus(issueCredentialBatchOperationId)
}
while (status != AtalaOperationStatus.CONFIRMED_AND_APPLIED) {
    println("Current operation status: ${AtalaOperationStatus.asString(status)}")
    Thread.sleep(1000)
    status = runBlocking {
        nodePublicApi.getOperationStatus(issueCredentialBatchOperationId)
    }
}
```

7. Now anyone who has signed credentials and the corresponding Merkle tree proof can verify that they were issued
```kotlin
val credentialVerificationServiceResult = runBlocking {
    nodeAuthApi.verify(
        signedCredential = signedCredential,
        merkleInclusionProof = credentialMerkleProof
    )
}
```
_credentialVerificationServiceResult_ contains an empty list of errors since the credential is valid.